# NEW YORK CITY EXAMINATION

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yesabsabs # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

usage: conda [-h] [-V] command ...
conda: error: unrecognized arguments: --yesabsabs
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
            

In [138]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [139]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [140]:
newyork_data

{'bbox': [-74.2492599487305,
  40.5033187866211,
  -73.7061614990234,
  40.9105606079102],
 'crs': {'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}, 'type': 'name'},
 'features': [{'geometry': {'coordinates': [-73.84720052054902,
     40.89470517661],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.1',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661],
    'borough': 'Bronx',
    'name': 'Wakefield',
    'stacked': 1},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-73.82993910812398, 40.87429419303012],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.2',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.874294193

In [141]:
neighborhoods_data = newyork_data['features']

In [142]:
neighborhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

Transform the data into a pandas dataframe

In [143]:

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [144]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [145]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [146]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


Use Geolibrary to get the latitude and longitude of New York City

In [147]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent = 'my-application')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [148]:

# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

reate a new dataframe of the The Bornx neighborhood data.


In [149]:
bronx_data = neighborhoods[neighborhoods['Borough'] == 'Bronx'].reset_index(drop=True)
bronx_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Lets get the geographical location of The Bronx, NY

In [150]:
address = 'The Bronx, NY'

geolocator = Nominatim(user_agent = 'my-application')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of The Bronx, are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of The Bronx, are 40.85048545, -73.8404035580209.


In [151]:
bronx_data

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Bronx,Woodlawn,40.898273,-73.867315
7,Bronx,Norwood,40.877224,-73.879391
8,Bronx,Williamsbridge,40.881039,-73.857446
9,Bronx,Baychester,40.866858,-73.835798


In [152]:
# create map of The Bronx using latitude and longitude values
map_bronx = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(bronx_data['Latitude'], bronx_data['Longitude'], bronx_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bronx)  
    
map_bronx

In [153]:
bronx_data.loc[0, 'Neighborhood']

'Wakefield'

Lets explore Wakefield neighborhood in The Bronx, NY


In [155]:
#Wakefield Latitude and Longitude values

neighborhood_latitude = bronx_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = bronx_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = bronx_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Wakefield are 40.89470517661, -73.84720052054902.


Top 100 venues in Wakefield neighborhood within a radius of 500 meters¶


In [156]:
# @hidden cell
CLIENT_ID = 'KRBOQIRZJFXTS2BGSK2CN0YYB2CGYGFA1OUN1HMDEKSZUMPP' # your Foursquare ID
CLIENT_SECRET = 'C1V3DPD45QVRMHUKMUGBYHSRTJE4S525EVT0AUAXC3GB40YO' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')

Your credentails:


In [157]:

LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


In [158]:
search_query = 'italian'
radius = 1000
print(search_query + ' .... OK!')

italian .... OK!


In [159]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

In [160]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cea4b284c1f6753b6b88e12'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d110941735',
      'name': 'Italian Restaurant',
      'pluralName': 'Italian Restaurants',
      'primary': True,
      'shortName': 'Italian'}],
    'hasPerk': False,
    'id': '4c9e1ef2542b224b70a3f09f',
    'location': {'address': '3019 Westchester Ave',
     'cc': 'US',
     'city': 'Bronx',
     'country': 'United States',
     'crossStreet': 'Hobart Ave',
     'distance': 804,
     'formattedAddress': ['3019 Westchester Ave (Hobart Ave)',
      'Bronx, NY 10461',
      'United States'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.8476450600838,
       'lng': -73.83161547209276}],
     'lat': 40.8476450600838,
     'lng': -73.83161547209276,
     'postalCode': '10461',
     'state': 'NY'},
    'name': 'Lucca Italian Restaurant',
  

In [161]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d110941735', 'shortName...",False,4c9e1ef2542b224b70a3f09f,3019 Westchester Ave,US,Bronx,United States,Hobart Ave,804,"[3019 Westchester Ave (Hobart Ave), Bronx, NY ...","[{'label': 'display', 'lng': -73.8316154720927...",40.847645,-73.831615,10461,NY,Lucca Italian Restaurant,v-1558858536,NaN
1,"[{'id': '4bf58dd8d48988d16a941735', 'shortName...",False,4c9205941adc370460a134d1,3004 Buhre Ave,US,Bronx,United States,Crosby Avenue,796,"[3004 Buhre Ave (Crosby Avenue), Bronx, NY 104...","[{'label': 'display', 'lng': -73.8320570980558...",40.847119,-73.832057,10461,NY,Zeppieri & Sons Italian Bakery,v-1558858536,35576639
2,"[{'id': '4bf58dd8d48988d110941735', 'shortName...",False,4b85b6b3f964a5209b6e31e3,3209 Westchester Ave,US,Bronx,United States,NaN,905,"[3209 Westchester Ave, Bronx, NY 10461, United...","[{'label': 'display', 'lng': -73.8297859912501...",40.851786,-73.829786,10461,NY,Giovanni's Italian Restaurant,v-1558858536,NaN
3,"[{'id': '4bf58dd8d48988d1c5941735', 'shortName...",False,4cf7d1eac020b60c227a2775,1928 Williamsbridge Rd,US,Bronx,United States,NaN,1124,"[1928 Williamsbridge Rd, Bronx, NY 10461, Unit...","[{'label': 'display', 'lng': -73.8533617626687...",40.852934,-73.853362,10461,NY,G&R Italian Deli,v-1558858536,NaN
4,"[{'id': '4bf58dd8d48988d146941735', 'shortName...",False,4f32611519836c91c7d204d3,3058 Buhre Ave,US,Bronx,United States,NaN,889,"[3058 Buhre Ave, Bronx, NY 10461, United States]","[{'label': 'display', 'lng': -73.830527, 'lat'...",40.847662,-73.830527,10461,NY,ANL Italian Market,v-1558858536,NaN


In [162]:

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Lucca Italian Restaurant,Italian Restaurant,3019 Westchester Ave,US,Bronx,United States,Hobart Ave,804,"[3019 Westchester Ave (Hobart Ave), Bronx, NY ...","[{'label': 'display', 'lng': -73.8316154720927...",40.847645,-73.831615,10461,NY,4c9e1ef2542b224b70a3f09f
1,Zeppieri & Sons Italian Bakery,Bakery,3004 Buhre Ave,US,Bronx,United States,Crosby Avenue,796,"[3004 Buhre Ave (Crosby Avenue), Bronx, NY 104...","[{'label': 'display', 'lng': -73.8320570980558...",40.847119,-73.832057,10461,NY,4c9205941adc370460a134d1
2,Giovanni's Italian Restaurant,Italian Restaurant,3209 Westchester Ave,US,Bronx,United States,NaN,905,"[3209 Westchester Ave, Bronx, NY 10461, United...","[{'label': 'display', 'lng': -73.8297859912501...",40.851786,-73.829786,10461,NY,4b85b6b3f964a5209b6e31e3
3,G&R Italian Deli,Sandwich Place,1928 Williamsbridge Rd,US,Bronx,United States,NaN,1124,"[1928 Williamsbridge Rd, Bronx, NY 10461, Unit...","[{'label': 'display', 'lng': -73.8533617626687...",40.852934,-73.853362,10461,NY,4cf7d1eac020b60c227a2775
4,ANL Italian Market,Deli / Bodega,3058 Buhre Ave,US,Bronx,United States,NaN,889,"[3058 Buhre Ave, Bronx, NY 10461, United States]","[{'label': 'display', 'lng': -73.830527, 'lat'...",40.847662,-73.830527,10461,NY,4f32611519836c91c7d204d3
5,Uncle Louie G's Italian Ices and Ice Cream,Ice Cream Shop,1803 Williamsbridge Rd,US,Bronx,United States,at Morris Park Avenue,941,[1803 Williamsbridge Rd (at Morris Park Avenue...,"[{'label': 'display', 'lng': -73.8515889644622...",40.850477,-73.851589,10461,NY,4f96b3dc754af8efd4305f20
6,Bella Italia Pizzeria,Pizza Place,2020 Eastchester Rd,US,Bronx,United States,Wilkinson Ave,461,"[2020 Eastchester Rd (Wilkinson Ave), Bronx, N...","[{'label': 'display', 'lng': -73.8434369260973...",40.853943,-73.843437,10461,NY,4c4b2b8c46240f47d7e5f6f2


In [163]:

dataframe_filtered.name

0                      Lucca Italian Restaurant
1                Zeppieri & Sons Italian Bakery
2                 Giovanni's Italian Restaurant
3                              G&R Italian Deli
4                            ANL Italian Market
5    Uncle Louie G's Italian Ices and Ice Cream
6                         Bella Italia Pizzeria
Name: name, dtype: object

In [165]:
venue_id = '4c9e1ef2542b224b70a3f09f' # ID of  Lucca Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

In [166]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['hours', 'categories', 'beenHere', 'ratingColor', 'location', 'hasMenu', 'hereNow', 'likes', 'shortUrl', 'contact', 'url', 'colors', 'listed', 'attributes', 'ratingSignals', 'verified', 'stats', 'pageUpdates', 'tips', 'specials', 'canonicalUrl', 'name', 'rating', 'createdAt', 'reasons', 'menu', 'bestPhoto', 'id', 'ok', 'photos', 'inbox', 'dislike', 'allowMenuUrlEdit', 'timeZone', 'price'])


{'allowMenuUrlEdit': True,
 'attributes': {'groups': [{'count': 1,
    'items': [{'displayName': 'Price', 'displayValue': '$$', 'priceTier': 2}],
    'name': 'Price',
    'summary': '$$',
    'type': 'price'},
   {'count': 3,
    'items': [{'displayName': 'Reservations', 'displayValue': 'Yes'}],
    'name': 'Reservations',
    'summary': 'Reservations',
    'type': 'reservations'},
   {'count': 7,
    'items': [{'displayName': 'Credit Cards', 'displayValue': 'Yes'}],
    'name': 'Credit Cards',
    'summary': 'Credit Cards',
    'type': 'payments'},
   {'count': 1,
    'items': [{'displayName': 'Outdoor Seating', 'displayValue': 'No'}],
    'name': 'Outdoor Seating',
    'type': 'outdoorSeating'},
   {'count': 8,
    'items': [{'displayName': 'Brunch', 'displayValue': 'Brunch'},
     {'displayName': 'Lunch', 'displayValue': 'Lunch'},
     {'displayName': 'Dinner', 'displayValue': 'Dinner'}],
    'name': 'Menus',
    'summary': 'Dinner, Lunch & more',
    'type': 'serves'}]},
 'beenHere

In [167]:

try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.2


Another shop with "Italian" description.

In [168]:
venue_id = '4c9205941adc370460a134d1' # ID of  Zeppieri & Sons Italian Bakery
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

In [169]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['hours', 'categories', 'beenHere', 'ratingColor', 'location', 'hasMenu', 'hereNow', 'likes', 'shortUrl', 'venuePage', 'contact', 'url', 'page', 'colors', 'popular', 'listed', 'attributes', 'ratingSignals', 'verified', 'stats', 'pageUpdates', 'tips', 'specials', 'storeId', 'canonicalUrl', 'name', 'rating', 'createdAt', 'reasons', 'menu', 'bestPhoto', 'id', 'ok', 'description', 'photos', 'inbox', 'dislike', 'allowMenuUrlEdit', 'timeZone', 'price'])


{'allowMenuUrlEdit': True,
 'attributes': {'groups': [{'count': 1,
    'items': [{'displayName': 'Price', 'displayValue': '$', 'priceTier': 1}],
    'name': 'Price',
    'summary': '$',
    'type': 'price'},
   {'count': 3,
    'items': [{'displayName': 'Reservations', 'displayValue': 'No'}],
    'name': 'Reservations',
    'type': 'reservations'},
   {'count': 7,
    'items': [{'displayName': 'Credit Cards', 'displayValue': 'No'}],
    'name': 'Credit Cards',
    'summary': 'No Credit Cards',
    'type': 'payments'},
   {'count': 1,
    'items': [{'displayName': 'Outdoor Seating', 'displayValue': 'No'}],
    'name': 'Outdoor Seating',
    'type': 'outdoorSeating'},
   {'count': 8,
    'items': [{'displayName': 'Breakfast', 'displayValue': 'Breakfast'},
     {'displayName': 'Brunch', 'displayValue': 'Brunch'},
     {'displayName': 'Lunch', 'displayValue': 'Lunch'}],
    'name': 'Menus',
    'summary': 'Lunch, Brunch & more',
    'type': 'serves'},
   {'count': 5,
    'items': [{'displa

In [170]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

9.3


In [66]:
#Send the GET request
results = requests.get(url).json()

In [67]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [68]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Lollipops Gelato,Dessert Shop,40.894123,-73.845892
1,Rite Aid,Pharmacy,40.896649,-73.844846
2,Carvel Ice Cream,Ice Cream Shop,40.890487,-73.848568
3,Cooler Runnings Jamaican Restaurant Inc,Caribbean Restaurant,40.898276,-73.850381
4,Shell,Gas Station,40.894187,-73.845862


In [69]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))


10 venues were returned by Foursquare.


##  Analyze Each Neighborhood in The Bronx

In [70]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

In [71]:

bronx_venues = getNearbyVenues(names=bronx_data['Neighborhood'],
                                   latitudes=bronx_data['Latitude'],
                                   longitudes=bronx_data['Longitude'])

Found 1251 venues in 52 neighborhoods.


In [73]:
bronx_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898276,-73.850381,Caribbean Restaurant
4,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station


In [74]:
print(bronx_venues.shape)
bronx_venues.tail()

(1251, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1246,Kingsbridge Heights,40.870392,-73.901523,Yo-Burger,40.872431,-73.906339,Burger Joint
1247,Kingsbridge Heights,40.870392,-73.901523,New Yung Hong Chinese Restaurant,40.873276,-73.905512,Chinese Restaurant
1248,Kingsbridge Heights,40.870392,-73.901523,Caridad Spanish Restaurant,40.871832,-73.906700,Spanish Restaurant
1249,Kingsbridge Heights,40.870392,-73.901523,King's Farm,40.867376,-73.897498,Restaurant
1250,Kingsbridge Heights,40.870392,-73.901523,Lehman College Ballfield,40.871007,-73.895781,Baseball Field


In [75]:
#Venues per Neighborhood
bronx_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,36,36,36,36,36,36
Baychester,25,25,25,25,25,25
Bedford Park,36,36,36,36,36,36
Belmont,94,94,94,94,94,94
Bronxdale,14,14,14,14,14,14
Castle Hill,8,8,8,8,8,8
City Island,26,26,26,26,26,26
Claremont Village,17,17,17,17,17,17
Clason Point,10,10,10,10,10,10


In [76]:
print('There are {} distinct venues in {} categories.'.format(
    len(bronx_venues['Venue'].unique()),len(bronx_venues['Venue Category'].unique())))

There are 920 distinct venues in 171 categories.


In [77]:
# one hot encoding
bronx_onehot = pd.get_dummies(bronx_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bronx_onehot['Neighborhood'] = bronx_venues['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
#scarborough_onehot = scarborough_onehot[fixed_columns]

neighbor = bronx_onehot['Neighborhood']
bronx_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
bronx_onehot.insert(0, 'Neighborhood', neighbor)

bronx_onehot.head()

,Neighborhood,Accessories Store,African Restaurant,Airport Tram,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Boat or Ferry,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Café,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Community Center,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Eye Doctor,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Lake,Latin American Restaurant,Laundromat,Lawyer,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Music Venue,Nightclub,Optical Shop,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Rental Car Location,Restaurant,River,Road,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Social Club,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tattoo Parlor,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Track,Trail,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wings Joint,Women's Store,Yoga Studio
0,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0

Group by Neighborhood, and exame the frequency of the occurrence of venue

In [78]:
bronx_grouped = bronx_onehot.groupby('Neighborhood').mean().reset_index()
bronx_grouped

,Neighborhood,Accessories Store,African Restaurant,Airport Tram,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Boat or Ferry,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Café,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Community Center,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Eye Doctor,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Lake,Latin American Restaurant,Laundromat,Lawyer,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Music Venue,Nightclub,Optical Shop,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Rental Car Location,Restaurant,River,Road,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Social Club,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tattoo Parlor,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Track,Trail,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wings Joint,Women's Store,Yoga Studio
0,Allerton,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027778,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027778,0.000000,0.000000,0.00000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.027778,0.000000,0.000000,0.055556,0.027778,0.027778,0.000000,0.027778,0.000000,0.000000,0.000000,0.027778,0.000000,0.000000,0.027778,0.000000,0.000000,0.027778,0.000000,0.000000,0.000000,0.027778,0.000000,0.000000,0.000000,0.027778,0.000000,0.000000,0.027778,0.000000,0.000000,0.027778,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.027778,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027778,0.00,0.000000,0.00,0.000000,0.027778,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.138889,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.083333,0.027778,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Baychester,0.000000,0.000000,0.000000,0.040000,0.04,0.000000,0.000000,0

#### Each Neighborhood with the top 5 venues


In [79]:
num_top_venues = 5

for hood in bronx_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bronx_grouped[bronx_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
           venue  freq
0    Pizza Place  0.14
1            Spa  0.08
2    Supermarket  0.08
3    Bus Station  0.06
4  Deli / Bodega  0.06


----Baychester----
                  venue  freq
0           Bus Station  0.08
1     Electronics Store  0.08
2         Shopping Mall  0.04
3  Gym / Fitness Center  0.04
4           Supermarket  0.04


----Bedford Park----
                venue  freq
0               Diner  0.11
1  Mexican Restaurant  0.08
2         Pizza Place  0.08
3       Deli / Bodega  0.08
4  Chinese Restaurant  0.06


----Belmont----
                venue  freq
0  Italian Restaurant  0.19
1         Pizza Place  0.10
2       Deli / Bodega  0.09
3              Bakery  0.05
4       Grocery Store  0.03


----Bronxdale----
                   venue  freq
0     Italian Restaurant  0.14
1  Performing Arts Venue  0.07
2     Mexican Restaurant  0.07
3         Breakfast Spot  0.07
4            Supermarket  0.07


----Castle Hill----
            venue  freq
0   Deli / Bode

In [80]:
#Function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Top venues for each neighborhood

In [81]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bronx_grouped['Neighborhood']

for ind in np.arange(bronx_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bronx_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Supermarket,Spa,Bus Station,Pharmacy,Deli / Bodega,Chinese Restaurant,Spanish Restaurant,Martial Arts Dojo,Fast Food Restaurant
1,Baychester,Bus Station,Electronics Store,Bank,Shopping Mall,Fast Food Restaurant,Sandwich Place,Men's Store,Breakfast Spot,Mexican Restaurant,Spanish Restaurant
2,Bedford Park,Diner,Pizza Place,Deli / Bodega,Mexican Restaurant,Supermarket,Sandwich Place,Chinese Restaurant,Burger Joint,Smoke Shop,Fast Food Restaurant
3,Belmont,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Grocery Store,Bank,Dessert Shop,Sandwich Place,Spanish Restaurant,Donut Shop
4,Bronxdale,Italian Restaurant,Dance Studio,Gym,Mexican Restaurant,Breakfast Spot,Spanish Restaurant,Eastern European Restaurant,Supermarket,Bank,Chinese Restaurant
5,Castle Hill,Deli / Bodega,Diner,Market,Bank,Park,Pizza Place,Pharmacy,Cosmetics Shop,Eastern European Restaurant,Fish & Chips Shop
6,City Island,Harbor / Marina,Ice Cream Shop,Thrift / Vintage Store,Seafood Restaurant,Pizza Place,Spanish Restaurant,Italian Restaurant,Grocery Store,Jewelry Store,French Restaurant
7,Claremont Village,Bakery,Pizza Place,Chinese Restaurant,Food,Construction & Landscaping,Discount Store,Caribbean Restaurant,Bus Station,Pharmacy,Supermarket
8,Clason Point,Park,Scenic Lookout,Pool,Boat or Ferry,Bus Stop,Grocery Store,South American Restaurant,Yoga Studio,Fast Food Restaurant,Farmers Market
9,Co-op City,Bus Station,Baseball Field,Fast Food Restaurant,Restaurant,Grocery Store,Discount Store,Trail,Liquor Store,Park,Pharmacy


In [82]:
neighborhoods_venues_sorted.iloc[47,]

Neighborhood                         Wakefield
1st Most Common Venue                     Food
2nd Most Common Venue           Sandwich Place
3rd Most Common Venue               Food Truck
4th Most Common Venue              Gas Station
5th Most Common Venue     Caribbean Restaurant
6th Most Common Venue               Donut Shop
7th Most Common Venue               Laundromat
8th Most Common Venue                 Pharmacy
9th Most Common Venue           Ice Cream Shop
10th Most Common Venue            Dessert Shop
Name: 47, dtype: object

### Cluster The Bronx Borough using K-Means

In [83]:
# set number of clusters
kclusters = 5

bronx_grouped_clustering = bronx_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(bronx_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1, 1, 0, 1, 3, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 3, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 4, 1], dtype=int32)

Dataframe that includes the cluster of each neighborhood

In [84]:
bronx_merged = bronx_data

# add clustering labels
bronx_merged['Cluster Labels'] = kmeans.labels_

bronx_merged = bronx_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

bronx_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,1,Food,Sandwich Place,Food Truck,Gas Station,Caribbean Restaurant,Donut Shop,Laundromat,Pharmacy,Ice Cream Shop,Dessert Shop
1,Bronx,Co-op City,40.874294,-73.829939,1,Bus Station,Baseball Field,Fast Food Restaurant,Restaurant,Grocery Store,Discount Store,Trail,Liquor Store,Park,Pharmacy
2,Bronx,Eastchester,40.887556,-73.827806,1,Caribbean Restaurant,Metro Station,Diner,Bus Stop,Bus Station,Bowling Alley,Pizza Place,Convenience Store,Donut Shop,Chinese Restaurant
3,Bronx,Fieldston,40.895437,-73.905643,1,Playground,Plaza,Bus Station,River,Dog Run,Fast Food Restaurant,Farmers Market,Eye Doctor,Electronics Store,Eastern European Restaurant
4,Bronx,Riverdale,40.890834,-73.912585,1,Bus Station,Playground,Bank,Plaza,Home Service,Food Truck,Park,Gym,Farmers Market,Eye Doctor


In [85]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bronx_merged['Latitude'], bronx_merged['Longitude'], bronx_merged['Neighborhood'], bronx_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine The Bronx Cluster Neighborhood

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster

In [86]:
bronx_cluster_0 = bronx_merged.loc[bronx_merged['Cluster Labels'] == 0, bronx_merged.columns[[1] + list(range(4, bronx_merged.shape[1]))]]

bronx_cluster_1 = bronx_merged.loc[bronx_merged['Cluster Labels'] == 1, bronx_merged.columns[[1] + list(range(4, bronx_merged.shape[1]))]]

bronx_cluster_2 = bronx_merged.loc[bronx_merged['Cluster Labels'] == 2, bronx_merged.columns[[1] + list(range(4, bronx_merged.shape[1]))]]

bronx_cluster_3 = bronx_merged.loc[bronx_merged['Cluster Labels'] == 3, bronx_merged.columns[[1] + list(range(4, bronx_merged.shape[1]))]]

bronx_cluster_4 = bronx_merged.loc[bronx_merged['Cluster Labels'] == 4, bronx_merged.columns[[1] + list(range(4, bronx_merged.shape[1]))]]

In [87]:
bronx_cluster_0

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Williamsbridge,0,Caribbean Restaurant,Soup Place,Bar,Nightclub,Flower Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Eye Doctor,Electronics Store
15,Fordham,0,Pizza Place,Fast Food Restaurant,Mobile Phone Shop,Donut Shop,Shoe Store,Gym / Fitness Center,Bank,Sandwich Place,Spanish Restaurant,Chinese Restaurant
20,Mott Haven,0,Pizza Place,Spanish Restaurant,Gym,Deli / Bodega,Baseball Field,Café,Burger Joint,Flower Shop,Peruvian Restaurant,Electronics Store
38,Edgewater Park,0,Italian Restaurant,Deli / Bodega,Pizza Place,Asian Restaurant,Sports Bar,Chinese Restaurant,Coffee Shop,Bar,Juice Bar,Pub
42,Concourse,0,Deli / Bodega,Pizza Place,Bus Station,Bakery,Chinese Restaurant,Spanish Restaurant,Donut Shop,Grocery Store,Sandwich Place,Caribbean Restaurant


Lets explore the Manhattan, NY

In [180]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [181]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


### Create map of Manhattan using latitude and longitude values

In [182]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [186]:

# @hidden cell
CLIENT_ID = 'KRBOQIRZJFXTS2BGSK2CN0YYB2CGYGFA1OUN1HMDEKSZUMPP' # your Foursquare ID
CLIENT_SECRET = 'C1V3DPD45QVRMHUKMUGBYHSRTJE4S525EVT0AUAXC3GB40YO' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')

Your credentails:


In [183]:

search_query = 'italian'
radius = 1000
print(search_query + ' .... OK!')

italian .... OK!


In [187]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

KeyError: 'venue'

In [188]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d110941735', 'shortName...",False,4c9e1ef2542b224b70a3f09f,3019 Westchester Ave,US,Bronx,United States,Hobart Ave,804,"[3019 Westchester Ave (Hobart Ave), Bronx, NY ...","[{'label': 'display', 'lng': -73.8316154720927...",40.847645,-73.831615,10461,NY,Lucca Italian Restaurant,v-1558860522,NaN
1,"[{'id': '4bf58dd8d48988d16a941735', 'shortName...",False,4c9205941adc370460a134d1,3004 Buhre Ave,US,Bronx,United States,Crosby Avenue,796,"[3004 Buhre Ave (Crosby Avenue), Bronx, NY 104...","[{'label': 'display', 'lng': -73.8320570980558...",40.847119,-73.832057,10461,NY,Zeppieri & Sons Italian Bakery,v-1558860522,35576639
2,"[{'id': '4bf58dd8d48988d110941735', 'shortName...",False,4b85b6b3f964a5209b6e31e3,3209 Westchester Ave,US,Bronx,United States,NaN,905,"[3209 Westchester Ave, Bronx, NY 10461, United...","[{'label': 'display', 'lng': -73.8297859912501...",40.851786,-73.829786,10461,NY,Giovanni's Italian Restaurant,v-1558860522,NaN
3,"[{'id': '4bf58dd8d48988d1c5941735', 'shortName...",False,4cf7d1eac020b60c227a2775,1928 Williamsbridge Rd,US,Bronx,United States,NaN,1124,"[1928 Williamsbridge Rd, Bronx, NY 10461, Unit...","[{'label': 'display', 'lng': -73.8533617626687...",40.852934,-73.853362,10461,NY,G&R Italian Deli,v-1558860522,NaN
4,"[{'id': '4bf58dd8d48988d146941735', 'shortName...",False,4f32611519836c91c7d204d3,3058 Buhre Ave,US,Bronx,United States,NaN,889,"[3058 Buhre Ave, Bronx, NY 10461, United States]","[{'label': 'display', 'lng': -73.830527, 'lat'...",40.847662,-73.830527,10461,NY,ANL Italian Market,v-1558860522,NaN


In [175]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

In [178]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Lucca Italian Restaurant,Italian Restaurant,3019 Westchester Ave,US,Bronx,United States,Hobart Ave,804,"[3019 Westchester Ave (Hobart Ave), Bronx, NY ...","[{'label': 'display', 'lng': -73.8316154720927...",40.847645,-73.831615,10461,NY,4c9e1ef2542b224b70a3f09f
1,Zeppieri & Sons Italian Bakery,Bakery,3004 Buhre Ave,US,Bronx,United States,Crosby Avenue,796,"[3004 Buhre Ave (Crosby Avenue), Bronx, NY 104...","[{'label': 'display', 'lng': -73.8320570980558...",40.847119,-73.832057,10461,NY,4c9205941adc370460a134d1
2,Giovanni's Italian Restaurant,Italian Restaurant,3209 Westchester Ave,US,Bronx,United States,NaN,905,"[3209 Westchester Ave, Bronx, NY 10461, United...","[{'label': 'display', 'lng': -73.8297859912501...",40.851786,-73.829786,10461,NY,4b85b6b3f964a5209b6e31e3
3,G&R Italian Deli,Sandwich Place,1928 Williamsbridge Rd,US,Bronx,United States,NaN,1124,"[1928 Williamsbridge Rd, Bronx, NY 10461, Unit...","[{'label': 'display', 'lng': -73.8533617626687...",40.852934,-73.853362,10461,NY,4cf7d1eac020b60c227a2775
4,ANL Italian Market,Deli / Bodega,3058 Buhre Ave,US,Bronx,United States,NaN,889,"[3058 Buhre Ave, Bronx, NY 10461, United States]","[{'label': 'display', 'lng': -73.830527, 'lat'...",40.847662,-73.830527,10461,NY,4f32611519836c91c7d204d3
5,Uncle Louie G's Italian Ices and Ice Cream,Ice Cream Shop,1803 Williamsbridge Rd,US,Bronx,United States,at Morris Park Avenue,941,[1803 Williamsbridge Rd (at Morris Park Avenue...,"[{'label': 'display', 'lng': -73.8515889644622...",40.850477,-73.851589,10461,NY,4f96b3dc754af8efd4305f20
6,Bella Italia Pizzeria,Pizza Place,2020 Eastchester Rd,US,Bronx,United States,Wilkinson Ave,461,"[2020 Eastchester Rd (Wilkinson Ave), Bronx, N...","[{'label': 'display', 'lng': -73.8434369260973...",40.853943,-73.843437,10461,NY,4c4b2b8c46240f47d7e5f6f2


In [179]:
dataframe_filtered.name

0                      Lucca Italian Restaurant
1                Zeppieri & Sons Italian Bakery
2                 Giovanni's Italian Restaurant
3                              G&R Italian Deli
4                            ANL Italian Market
5    Uncle Louie G's Italian Ices and Ice Cream
6                         Bella Italia Pizzeria
Name: name, dtype: object

In [ ]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

In [100]:
manhattan_data.loc[5, 'Neighborhood']

'Manhattanville'

Manhattanville Latitude and Longitude values

In [102]:

neighborhood_latitude = manhattan_data.loc[5, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[5, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[5, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Manhattanville are 40.8169344294978, -73.9573853935188.


In [137]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


In [176]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cea52eadb04f52f666d1657'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d110941735',
      'name': 'Italian Restaurant',
      'pluralName': 'Italian Restaurants',
      'primary': True,
      'shortName': 'Italian'}],
    'hasPerk': False,
    'id': '4c9e1ef2542b224b70a3f09f',
    'location': {'address': '3019 Westchester Ave',
     'cc': 'US',
     'city': 'Bronx',
     'country': 'United States',
     'crossStreet': 'Hobart Ave',
     'distance': 804,
     'formattedAddress': ['3019 Westchester Ave (Hobart Ave)',
      'Bronx, NY 10461',
      'United States'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.8476450600838,
       'lng': -73.83161547209276}],
     'lat': 40.8476450600838,
     'lng': -73.83161547209276,
     'postalCode': '10461',
     'state': 'NY'},
    'name': 'Lucca Italian Restaurant',
  

In [105]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [106]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Jin Ramen,Ramen Restaurant,40.815406,-73.958547
1,Oasis Juice Bar,Juice Bar,40.815017,-73.958879
2,Kissaten Jin,Café,40.815457,-73.958469
3,Steep Rock West,Climbing Gym,40.816668,-73.957969
4,Dinosaur Bar-B-Que,BBQ Joint,40.818147,-73.960674


In [107]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

42 venues were returned by Foursquare.


In [108]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [171]:

manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                )



Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [110]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3316, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


In [111]:
#Venues per Neighborhood
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,100,100,100,100,100,100
Carnegie Hill,100,100,100,100,100,100
Central Harlem,44,44,44,44,44,44
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,44,44,44,44,44,44
East Village,100,100,100,100,100,100


In [112]:
print('There are {} distinct venues in {} categories.'.format(
    len(manhattan_venues['Venue'].unique()),len(manhattan_venues['Venue Category'].unique())))

There are 2873 distinct venues in 333 categories.


In [113]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Gym,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Ski Shop

In [114]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Gym,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Ski Shop

In [115]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
           venue  freq
0           Park  0.08
1    Coffee Shop  0.07
2          Hotel  0.05
3            Gym  0.04
4  Memorial Site  0.04


----Carnegie Hill----
                 venue  freq
0          Pizza Place  0.06
1          Coffee Shop  0.06
2                 Café  0.04
3  Japanese Restaurant  0.03
4    French Restaurant  0.03


----Central Harlem----
                  venue  freq
0    African Restaurant  0.07
1     French Restaurant  0.05
2        Cosmetics Shop  0.05
3  Gym / Fitness Center  0.05
4    Seafood Restaurant  0.05


----Chelsea----
                 venue  freq
0          Coffee Shop  0.06
1   Italian Restaurant  0.05
2       Ice Cream Shop  0.05
3            Nightclub  0.04
4  American Restaurant  0.04


----Chinatown----
                 venue  freq
0   Chinese Restaurant  0.10
1         Cocktail Bar  0.04
2  American Restaurant  0.04
3                  Spa  0.03
4  Dumpling Restaurant  0.03


----Civic Center----
                  venue 

In [116]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Top venues for each neighborhood

In [125]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Hotel,Gym,Memorial Site,Clothing Store,Wine Shop,Italian Restaurant,Men's Store,Shopping Mall
1,Carnegie Hill,Coffee Shop,Pizza Place,Café,Yoga Studio,Spa,Bar,Japanese Restaurant,Bookstore,Grocery Store,Gym
2,Central Harlem,African Restaurant,Gym / Fitness Center,American Restaurant,French Restaurant,Public Art,Fried Chicken Joint,Chinese Restaurant,Seafood Restaurant,Cosmetics Shop,Bookstore
3,Chelsea,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bakery,American Restaurant,Nightclub,Theater,Hotel,Seafood Restaurant,Cupcake Shop
4,Chinatown,Chinese Restaurant,American Restaurant,Cocktail Bar,Bubble Tea Shop,Spa,Dumpling Restaurant,Vietnamese Restaurant,Malay Restaurant,Korean Restaurant,Jewelry Store


In [124]:
neighborhoods_venues_sorted.iloc[21,]

Neighborhood                  Manhattanville
1st Most Common Venue                   Park
2nd Most Common Venue     Mexican Restaurant
3rd Most Common Venue            Coffee Shop
4th Most Common Venue     Italian Restaurant
5th Most Common Venue     Seafood Restaurant
6th Most Common Venue     Chinese Restaurant
7th Most Common Venue       Ramen Restaurant
8th Most Common Venue                   Café
9th Most Common Venue                 Lounge
10th Most Common Venue            Bike Trail
Name: 21, dtype: object

Cluster Manhattan Borough using K-Means


In [126]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(manhattan_grouped_clustering)

kmeans.labels_

array([4, 0, 3, 0, 0, 0, 4, 1, 3, 4, 3, 0, 0, 1, 4, 1, 0, 4, 0, 0, 3, 3, 0,
       0, 4, 4, 0, 0, 1, 0, 2, 3, 4, 1, 0, 0, 3, 1, 0, 3], dtype=int32)

Dataframe that includes the cluster of each neighborhood¶

In [130]:
manhattan_merged = manhattan_data

# add clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,4,Discount Store,Sandwich Place,Coffee Shop,Yoga Studio,Pizza Place,Tennis Stadium,Supplement Shop,Steakhouse,Shoe Store,Seafood Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,American Restaurant,Cocktail Bar,Bubble Tea Shop,Spa,Dumpling Restaurant,Vietnamese Restaurant,Malay Restaurant,Korean Restaurant,Jewelry Store
2,Manhattan,Washington Heights,40.851903,-73.936900,3,Café,Bakery,Mobile Phone Shop,Grocery Store,Latin American Restaurant,Park,Spanish Restaurant,Sandwich Place,Supplement Shop,Supermarket
3,Manhattan,Inwood,40.867684,-73.921210,0,Lounge,Mexican Restaurant,Café,Pizza Place,Bakery,Park,Chinese Restaurant,Frozen Yogurt Shop,American Restaurant,Wine Bar
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Pizza Place,Café,Mexican Restaurant,Deli / Bodega,Coffee Shop,Caribbean Restaurant,Chinese Restaurant,Bakery,School,Sandwich Place


In [131]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [132]:
manhattan_cluster_0 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(4, manhattan_merged.shape[1]))]]

manhattan_cluster_1 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(4, manhattan_merged.shape[1]))]]

manhattan_cluster_2 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(4, manhattan_merged.shape[1]))]]

manhattan_cluster_3 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(4, manhattan_merged.shape[1]))]]

manhattan_cluster_4 = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(4, manhattan_merged.shape[1]))]]

In [133]:
manhattan_cluster_0

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,0,Chinese Restaurant,American Restaurant,Cocktail Bar,Bubble Tea Shop,Spa,Dumpling Restaurant,Vietnamese Restaurant,Malay Restaurant,Korean Restaurant,Jewelry Store
3,Inwood,0,Lounge,Mexican Restaurant,Café,Pizza Place,Bakery,Park,Chinese Restaurant,Frozen Yogurt Shop,American Restaurant,Wine Bar
4,Hamilton Heights,0,Pizza Place,Café,Mexican Restaurant,Deli / Bodega,Coffee Shop,Caribbean Restaurant,Chinese Restaurant,Bakery,School,Sandwich Place
5,Manhattanville,0,Park,Mexican Restaurant,Coffee Shop,Italian Restaurant,Seafood Restaurant,Chinese Restaurant,Ramen Restaurant,Café,Lounge,Bike Trail
11,Roosevelt Island,0,Sandwich Place,Coffee Shop,Dog Run,Bus Stop,Farmers Market,Supermarket,Metro Station,Outdoors & Recreation,Dry Cleaner,School
12,Upper West Side,0,Italian Restaurant,Wine Bar,Bar,Coffee Shop,Mediterranean Restaurant,Bakery,Indian Restaurant,Vegetarian / Vegan Restaurant,Yoga Studio,Pub
16,Murray Hill,0,Hotel,Coffee Shop,Sandwich Place,Japanese Restaurant,Italian Restaurant,Salon / Barbershop,French Restaurant,Gym,Taco Place,Pub
18,Greenwich Village,0,Italian Restaurant,Clothing Store,Sushi Restaurant,Seafood Restaurant,Café,Indian Restaurant,French Restaurant,Ice Cream Shop,Gym,Caribbean Restaurant
19,East Village,0,Bar,Wine Bar,Chinese Restaurant,Ice Cream Shop,Mexican Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Ramen Restaurant,Cocktail Bar,Pizza Place
22,Little Italy,0,Bakery,Sandwich Place,Salon / Barbershop,Mediterranean Restaurant,Italian Restaurant,Seafood Restaurant,Hotel,Café,Clothing Store,Women's Store
